## <center style="color:Blue">World Happiness Dataset with Spark</center>

<center><img src="https://steemitimages.com/1280x0/https://www.psychologies.co.uk/sites/default/files/styles/psy2_page_header/public/wp-content/uploads/2012/03/happy.jpg"></center>

# <center>Importing spark and data</center>


In [ ]:
!pip install pyspark #install pySpark

In [ ]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
spark= SparkSession.builder.getOrCreate()

In [ ]:
path= "../input/world-happiness-report-2021/world-happiness-report.csv"
data= spark.read.csv(path)

In [ ]:
data.show(10)

## <center>Let's make it looks better</center>

In [ ]:
data.printSchema()

## 1.1 Creat a schema 

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

In [ ]:
new_schema= StructType([StructField("Country", StringType(), True),
                        StructField("Year", StringType(), True),
                        StructField("Life_Ladder", StringType(), True),
                        StructField("Log_Gdp", StringType(), True),
                        StructField("Social_support", StringType(), True),
                        StructField("Healthy_life_expectancy", StringType(), True),
                        StructField("Freedom_to_make_life_choices ", StringType(), True),
                        StructField("Generosity", StringType(), True),
                        StructField("Perceptions_of_corruption", StringType(), True),
                        StructField("Positive_affect", StringType(), True),
                        StructField("Negative_affect", StringType(), True)
                       ])

In [ ]:
data= spark.read.csv(path, schema=new_schema)
data.show(3)

## 1.2 Changing datatype

In [ ]:
for i in data.columns:
    if i != "Country":
        data= data.withColumn(i+"_",data[i].cast(FloatType())).drop(i)
data= data.withColumn('Year',data['Year_'].cast(IntegerType())).drop("Year_")

In [ ]:
data.printSchema()

In [ ]:
data.show(5)

In [ ]:
data.show(4)

## 1.3 Drop null values

In [ ]:
print(data.count())
data=data.na.drop()
data.show(5)
print(data.count())

# <center> Data Exploration </center>

## Question2: How many Country there are?

In [ ]:
country_data = data.groupBy('Country').count().show()

## Question2: How many data rows collected from Afghanistan?

In [ ]:
Af_data= data.filter(data['Country']=="Afghanistan")
Af_data= Af_data.drop('Country')
print("Number of rows that collected from Afghanistan =", Af_data.count())
Af_data.show(12)

## Question3: what is tha average of Positive and Negative affect in Afghanistan?

In [ ]:
Af_data.groupBy().avg('Positive_affect_').show()
Af_data.groupBy().avg('Negative_affect_').show()

# <center>Adding 2021 Dataset to the main data</center>

In [ ]:
path_2021= "../input/world-happiness-report-2021/world-happiness-report-2021.csv"
data_2021= spark.read.csv(path_2021, schema=new_schema)

In [ ]:
data_2021.show(3)

In [ ]:
for i in data_2021.columns:
    if i != "Country":
        data_2021= data_2021.withColumn(i+"_",data_2021[i].cast(FloatType())).drop(i)
data_2021= data_2021.withColumn('Year',data_2021['Year_'].cast(IntegerType())).drop("Year_")


In [ ]:
data_2021.show(10)

In [ ]:
all_data= data.union(data_2021)

In [ ]:
all_data.show(5)